In [1]:
import os
import tarfile
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
import librosa
import numpy as np
import pandas as pd
from scipy import stats
import cupy as cp
from tqdm import tqdm

In [2]:
BASE_URL = "http://www.repository.voxforge1.org/downloads/SpeechCorpus/Trunk/Audio/Main/16kHz_16bit/"
DOWNLOAD_DIR = "voxforge_data"
EXTRACT_DIR = "voxforge_data/extracted"
RESULTS_FILE = "voxforge_data/results_summary.txt"
FEATURES_FILE = "voxforge_data/audio_features.parquet"

In [3]:
os.makedirs(DOWNLOAD_DIR, exist_ok=True)
os.makedirs(EXTRACT_DIR, exist_ok=True)

# Download & Extract

In [4]:
### 1. Get .tgz links from base URL
def get_tgz_links():
    response = requests.get(BASE_URL)
    if response.status_code != 200:
        print("Failed to retrieve webpage")
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    pre_tag = soup.find("pre")
    if not pre_tag:
        print("Could not find the <pre> tag.")
        return []
    
    return [BASE_URL + a["href"] for a in pre_tag.find_all("a", href=True) if a["href"].endswith(".tgz")]

### 2. Download .tgz files in parallel
def download_tgz(url):
    filename = os.path.join(DOWNLOAD_DIR, url.split("/")[-1])
    if os.path.exists(filename):
        print(f"{filename} already exists, skipping download.")
        return filename

    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(filename, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Downloaded {filename}")
        return filename
    print(f"Failed to download {url}")
    return None

def parallel_download_tgz():
    tgz_links = get_tgz_links()
    with ThreadPoolExecutor(max_workers=12) as executor:
        return list(executor.map(download_tgz, tgz_links))

### 3. Extract .tgz files in parallel
def extract_tgz(filepath):
    with tarfile.open(filepath, "r:gz") as tar:
        tar.extractall(path=EXTRACT_DIR)
    print(f"Extracted {filepath}")

def parallel_extract_tgz(tgz_files):
    with ThreadPoolExecutor(max_workers=8) as executor:
        executor.map(extract_tgz, tgz_files)
    print("All files extracted.")

### 4. Filter .wav files by human vocal range
def is_within_vocal_range(wav_path):
    y, sr = librosa.load(wav_path, sr=None)
    freqs = np.fft.rfftfreq(len(y), d=1/sr)
    spectrum = np.abs(np.fft.rfft(y))

    human_vocal_range = (0, 280)
    valid_freqs = freqs[(freqs >= human_vocal_range[0]) & (freqs <= human_vocal_range[1])]
    valid_spectrum = spectrum[:len(valid_freqs)]
    return valid_spectrum.any()

def get_vocal_range_wavs():
    
    """Parallelized version of get_vocal_range_wavs() with progress updates."""
    all_wavs = []
    for root, _, files in os.walk(EXTRACT_DIR):
        for file in files:
            if file.endswith(".wav"):
                all_wavs.append(os.path.join(root, file))

    print(f"Total .wav files found: {len(all_wavs)}")

    valid_wavs = []
    
    # Process in parallel with tqdm for real-time feedback
    with ProcessPoolExecutor(max_workers=10) as executor:
        results = list(tqdm(executor.map(is_within_vocal_range, all_wavs), total=len(all_wavs), desc="Filtering .wav files"))

    # Collect only the valid files
    valid_wavs = [wav for wav, valid in zip(all_wavs, results) if valid]

    print(f"\nFiltered to {len(valid_wavs)} .wav files within human vocal range.")
    print(f"Dropped files: {len(all_wavs) - len(valid_wavs)}")
    
    return valid_wavs

In [5]:
downloaded_tgz_files = parallel_download_tgz()
parallel_extract_tgz(downloaded_tgz_files)

voxforge_data/1028-20100710-hne.tgz already exists, skipping download.
voxforge_data/1337ad-20170321-ajg.tgz already exists, skipping download.
voxforge_data/1337ad-20170321-tkg.tgz already exists, skipping download.
voxforge_data/1337ad-20170321-ynk.tgz already exists, skipping download.
voxforge_data/1snoke-20120412-hge.tgz already exists, skipping download.
voxforge_data/23yipikaye-20100807-ujm.tgz already exists, skipping download.
voxforge_data/2old2play-20110606-hcn.tgz already exists, skipping download.
voxforge_data/2old2play-20110606-wlt.tgz already exists, skipping download.
voxforge_data/314piwm-20130617-xuo.tgz already exists, skipping download.
voxforge_data/AJN-20131002-zcl.tgz already exists, skipping download.
voxforge_data/AT-20130718-lws.tgz already exists, skipping download.
voxforge_data/Aaron-20080318-kdl.tgz already exists, skipping download.
voxforge_data/Aaron-20080318-lbb.tgz already exists, skipping download.
voxforge_data/Aaron-20080318-lbk.tgz already exists

In [5]:
wav_files = get_vocal_range_wavs()
print(f"Found {len(wav_files)} .wav files in the human vocal range.")


Total .wav files found: 87421


Filtering .wav files: 100%|██████████████████████████████████████████████████████| 87421/87421 [03:51<00:00, 376.98it/s]



Filtered to 87421 .wav files within human vocal range.
Dropped files: 0
Found 87421 .wav files in the human vocal range.


# Filtering

In [6]:
import random

sample_wavs = random.sample(wav_files, 383)

# Init lists
out_of_range_ratios = []
in_range_energies = []
out_range_energies = []

for wav_path in tqdm(sample_wavs, desc="Analyzing sampled .wav files"):
    y, sr = librosa.load(wav_path, sr=None)
    freqs = np.fft.rfftfreq(len(y), d=1/sr)
    spectrum = np.abs(np.fft.rfft(y)) ** 2  # Power spectrum

    in_energy = np.sum(spectrum[freqs <= 280])
    out_energy = np.sum(spectrum[freqs > 280])
    total_energy = in_energy + out_energy

    if total_energy == 0:
        ratio = 0.0
    else:
        ratio = out_energy / total_energy

    in_range_energies.append(in_energy)
    out_range_energies.append(out_energy)
    out_of_range_ratios.append(ratio)

series = pd.Series(out_of_range_ratios, name="out_of_range_ratio")
print(series.describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95]))

Analyzing sampled .wav files: 100%|██████████████████████████████████████████████████| 383/383 [00:02<00:00, 141.62it/s]

count    383.000000
mean       0.581168
std        0.268646
min        0.001707
25%        0.423423
50%        0.638410
75%        0.788002
90%        0.880268
95%        0.919165
max        0.996774
Name: out_of_range_ratio, dtype: float64


In [7]:
def compute_ratio(wav_path):
    y, sr = librosa.load(wav_path, sr=None)
    spectrum = np.abs(np.fft.rfft(y))
    freqs = np.fft.rfftfreq(len(y), d=1/sr)

    in_range = (freqs >= 0) & (freqs <= 280)
    energy_in_range = np.sum(spectrum[in_range])
    energy_total = np.sum(spectrum)
    ratio = 1 - (energy_in_range / energy_total) if energy_total > 0 else 1
    return wav_path, ratio


def filter_by_out_of_range_ratio(wav_paths, threshold):
    """Filter .wav files where too much energy is outside the 0–280 Hz human vocal range."""
    print("Calculating out-of-range energy ratios...")

    with ProcessPoolExecutor(max_workers=10) as executor:
        results = list(tqdm(executor.map(compute_ratio, wav_paths), total=len(wav_paths)))

    filtered = [wav for wav, ratio in results if ratio < threshold]
    dropped = [wav for wav, ratio in results if ratio >= threshold]

    print(f"Kept {len(filtered)} WAV files below threshold ({threshold}).")
    print(f"Dropped {len(dropped)} WAV files due to high out-of-range energy.")
    return filtered



In [8]:
filtered_wavs = filter_by_out_of_range_ratio(wav_files, threshold=0.9)

Calculating out-of-range energy ratios...


100%|████████████████████████████████████████████████████████████████████████████| 87421/87421 [03:58<00:00, 367.12it/s]


Kept 69038 WAV files below threshold (0.9).
Dropped 18383 WAV files due to high out-of-range energy.


# Space for trimming if needed

# Extract Features & Save as Parquet

In [9]:
def compute_fft_features_cpu(wav_path):
    """Extract FFT spectrum and frequencies using NumPy. Returns (filename, freqs, spectrum)."""
    y, sr = librosa.load(wav_path, sr=None)
    spectrum = np.abs(np.fft.rfft(y))
    freqs = np.fft.rfftfreq(len(y), d=1/sr)
    
    rel_path = os.path.relpath(wav_path, EXTRACT_DIR)
    return rel_path, freqs, spectrum

def compute_statistics(item):
    """Takes (filename, freqs, spectrum) and returns computed audio features."""
    filename, freqs, spectrum = item

    if len(spectrum) == 0:
        return None

    # Normalize spectrum for spectral entropy and flatness
    spectrum_norm = spectrum / np.sum(spectrum) if np.sum(spectrum) > 0 else np.zeros_like(spectrum)

    # Core stats
    mean_freq = np.mean(freqs) / 1000
    std_freq = np.std(freqs) / 1000
    median_freq = np.median(freqs) / 1000
    first_quantile = np.percentile(freqs, 25) / 1000
    third_quantile = np.percentile(freqs, 75) / 1000
    iqr = third_quantile - first_quantile
    skewness = stats.skew(spectrum)
    kurtosis = stats.kurtosis(spectrum)
    mode_freq = freqs[np.argmax(spectrum)] / 1000
    peak_freq = mode_freq

    # Spectral entropy
    sp_entropy = -np.sum(spectrum_norm * np.log2(spectrum_norm + 1e-10))

    # Spectral flatness (geometric mean / arithmetic mean)
    flatness = stats.gmean(spectrum + 1e-10) / (np.mean(spectrum) + 1e-10)

    # Centroid
    centroid = np.sum(freqs * spectrum) / np.sum(spectrum) / 1000 if np.sum(spectrum) > 0 else 0

    # Modulation index: rough approximation as std/sum
    modindx = np.std(spectrum) / (np.mean(spectrum) + 1e-10)

    return [
        filename, mean_freq, std_freq, median_freq,
        first_quantile, third_quantile, iqr,
        skewness, kurtosis, mode_freq, peak_freq,
        sp_entropy, flatness, centroid, modindx
    ]


def parallel_extract_audio_features(wav_files, batch_size=5000):
    """Extract audio features in batches to avoid overwhelming memory."""
    if not wav_files:
        print("No valid .wav files found for feature extraction.")
        return

    total = len(wav_files)
    print(f"Step 1: Computing FFT (CPU) for {total} .wav files in batches of {batch_size}...")

    all_results = []

    for i in range(0, total, batch_size):
        batch_files = wav_files[i:i + batch_size]
        print(f"\nProcessing batch {i // batch_size + 1} / {(total - 1) // batch_size + 1} ({len(batch_files)} files)")

        fft_outputs = []
        for wav in tqdm(batch_files, desc="Computing FFT (CPU)", leave=False):
            try:
                fft_outputs.append(compute_fft_features_cpu(wav))
            except Exception as e:
                print(f"Error processing {wav}: {e}")

        print("Step 2: Computing statistics (parallel CPU)...")
        with ProcessPoolExecutor(max_workers=10) as executor:
            batch_results = list(tqdm(executor.map(compute_statistics, fft_outputs),
                                      total=len(fft_outputs), desc="Computing Stats", leave=False))

        batch_results = [r for r in batch_results if r is not None]
        all_results.extend(batch_results)

    df = pd.DataFrame(all_results, columns=[
    "filename", "mean_freq_kHz", "std_freq_kHz", "median_freq_kHz",
    "first_quantile_kHz", "third_quantile_kHz", "iqr_kHz",
    "skewness", "kurtosis", "mode_freq_kHz", "peak_freq_kHz",
    "sp_entropy", "flatness", "centroid_kHz", "modindx"
    ])

    df.to_parquet(FEATURES_FILE)
    print(f"\n✅ Feature extraction complete. Saved to {FEATURES_FILE}.")



In [10]:
parallel_extract_audio_features(filtered_wavs)

Step 1: Computing FFT (CPU) for 69038 .wav files in batches of 5000...

Processing batch 1 / 14 (5000 files)


Step 2: Computing statistics (parallel CPU)...



Processing batch 2 / 14 (5000 files)


Step 2: Computing statistics (parallel CPU)...



Processing batch 3 / 14 (5000 files)


...


Processing batch 13 / 14 (5000 files)


Step 2: Computing statistics (parallel CPU)...



Processing batch 14 / 14 (4038 files)


Step 2: Computing statistics (parallel CPU)...



✅ Feature extraction complete. Saved to voxforge_data/audio_features.parquet.


In [11]:
df = pd.read_parquet("voxforge_data/audio_features.parquet")

In [12]:
df.head(30)

,filename,mean_freq_kHz,std_freq_kHz,median_freq_kHz,first_quantile_kHz,third_quantile_kHz,iqr_kHz,skewness,kurtosis,mode_freq_kHz,peak_freq_kHz,sp_entropy,flatness,centroid_kHz,modindx
0,robin-20070310-vf12/wav/vf12-34.wav,3.999961,2.309424,3.999961,1.999981,5.999942,3.999961,4.289625,22.129724,0.596192,0.596192,13.797732,0.131873,1.104101,2.316162
1,robin-20070310-vf12/wav/vf12-29.wav,4.000000,2.309454,4.000000,2.000000,6.000000,4.000000,4.468912,24.387798,0.518989,0.518989,13.671813,0.145211,1.173674,2.266816
2,robin-20070310-vf12/wav/vf12-24.wav,4.000000,2.309442,4.000000,2.000000,6.000000,4.000000,4.768208,28.682903,0.522362,0.522362,13.983390,0.154861,1.182493,2.340302
3,robin-20070310-vf12/wav/vf12-06.wav,4.000000,2.309460,4.000000,2.000000,6.000000,4.000000,4.288241,22.928845,0.165990,0.165990,13.383058,0.113975,1.056596,2.309039
4,robin-20070310-vf12/wav/vf12-38.wav,3.999964,2.309422,3.999964,1.999982,5.999946,3.999964,4.437734,23.512740,0.327474,0.327474,14.031024,0.154637,1.212154,2.255088
5,robin-20070310-vf12/wav/vf12-03.wav,4.000000,2.309478,4.000000,2.000000,6.000000,4.000000,5.017607,32.963787,0.439637,0.439637,12.726298,0.107020,0.936312,2.643977
6,robin-20070310-vf12/wav/vf12-40.wav,4.000000,2.309468,4.000000,2.000000,6.000000,4.000000,6.397723,66.214303,0.586054,0.586054,13.176128,0.135573,1.121801,2.504664
7,robin-20070310-vf12/wav/vf12-32.wav,4.000000,2.309434,4.000000,2.000000,6.000000,4.000000,5.048145,31.834775,0.513476,0.513476,14.314123,0.160156,1.210400,2.406112
8,robin-20070310-vf12/wav/vf12-09.wav,3.999960,2.309424,3.999960,1.999980,5.999940,3.999960,5.452632,43.163835,0.550339,0.550339,13.920943,0.164200,1.255173,2.273952
9,robin-20070310-vf12/wav/vf12-19.wav,4.000000,2.309475,4.000000,2.000000,6.000000,4.000000,4.593074,25.416905,0.348617,0.348617,13.171095,0.156655,1.203686,2.308686
